# OpenAI APIの前準備

In [ ]:
# パッケージのインストール
!pip install openai

In [ ]:
# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI

# クライアントの準備
client = OpenAI()

In [ ]:
import json

# JSON出力ヘルパーの準備
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# アシスタント

In [ ]:
# アシスタントの作成
assistant = client.beta.assistants.create(
    name="お天気ボット",
    instructions="あなたはお天気ボットです。 提供されている関数を使用して質問に答えます。",
    model="gpt-4-1106-preview",
    tools=[{
        "type": "function",
        "function": {
            "name": "getCurrentTemperature",
            "description": "特定の場所の気温を取得",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "場所 (例:東京)"},
                    "unit": {"type": "string", "enum": ["c", "f"]}
                },
                "required": ["location"]
            }
        }
    }]
)

In [ ]:
# スレッドの作成
thread = client.beta.threads.create()

In [ ]:
# ユーザーメッセージの追加
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="群馬県の気温を教えてください。"
)

In [ ]:
# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)

user : 群馬県の気温を教えてください。


In [ ]:
# アシスタントの実行
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [ ]:
# Runのステータスの確認
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(run.status)

requires_action


In [ ]:
import json

# require_actionのパラメータの取得
tool_id = run.required_action.submit_tool_outputs.tool_calls[0].id
tool_function_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
tool_function_arguments = json.loads(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)

print("id:", tool_id)
print("name:", tool_function_name)
print("arguments:", tool_function_arguments)

id: call_eOhO9dbeVVskFkRpoDe2AdLX
name: getCurrentTemperature
arguments: {'location': '群馬県', 'unit': 'c'}


In [ ]:
# 特定の場所の気温を取得する関数 (ダミー)
def getCurrentTemperature(location, unit):
    print("location:", location)
    print("unit:", unit)
    return "22"

# 特定の場所の気温を取得
tool_function_output = getCurrentTemperature(
    tool_function_arguments["location"],
    tool_function_arguments["unit"]
)

location: 群馬県
unit: c


In [ ]:
# 関数呼び出し結果のアシスタントへの提出
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_id,
            "output": tool_function_output,
        }
    ]
)

In [ ]:
# Runのステータスの確認
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(run.status)

In [ ]:
# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)